In [ ]:
%load_ext autoreload
%autoreload 2

This notebook makes plots to show the shape similarities of different dbs points.

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import zfit
from zfit import z
import re
from scipy.optimize import curve_fit

In [ ]:
import mplhep as hep
hep.style.use(hep.style.CMS)
plt.rcParams.update({
    "text.usetex": True,
})

In [ ]:
from plotting_meta.plotting_meta import bins, binning_type, cms_format_fig, Bins

from bff_plotting_tools.make_hists import make_sys, SysHist
from bff_plotting_tools.data_getter import get_data
from bff_plotting_tools.utils import hist_chi2, chi_squared

from bff_signal_model.bff_signal_model import bff_signal_model, reset_params, mu, sys_func, sys_func_offset, popt_sys_dict
from bff_signal_model.utils import fit_model

from bff_processor.plotting_utils import color_fader
from bff_processor.utils import constant

In [ ]:
era = '2016'
df, lumi = get_data(era)
compute_hesse=True
masses = np.unique(df[df.type=='sig'].mass)
masses = [mass for mass in masses if mass!=200]
masses 

In [ ]:
obs = zfit.Space("x", limits=bins.calc_bin_range())

In [ ]:
hists= pd.DataFrame([{"reg":reg, "mass":int(mass), "dbs":dbs} for reg in ['SR1','SR2'] for mass in masses for dbs in [0.04, 0.5, 1.0]])

In [ ]:
# make histograms
hist_dict = {}
for i, row in hists.iterrows():
    reg, mass, dbs = row
    if mass not in [125,200, 350, 500]: continue
    # get hists
    sig_df = df[(df.mass==mass) & (df.dbs==dbs)]
    data = sig_df.DiLepMass.to_numpy()
    weight = sig_df.Weight.to_numpy()
    #fit signal
    mu.set_value(mass)
    sm = bff_signal_model(obs, mu=mu)
    width = sm.sigma().numpy()
    mean = mu.numpy()
    print(mean, width)
    print(reg, mass, dbs)
    bins_centered = Bins(np.linspace(mean-width*7, mean+7*width,50))
    if sig_df.shape[0]==0: continue
    hist = make_sys(sig_df, 'DiLepMass', reg, bin_edges=bins_centered.bin_edges)
    #compute needed quantities
    normalization = np.sum(hist.nominal)
    results_fit = fit_model(obs, data, weight, sm, compute_hesse=True)
    popt_sys = popt_sys_dict[era][reg]
    sys_width = sys_func_offset(row, .01, *popt_sys)
    print(sys_width, results_fit)
    fit_hist = sm.make_hist(bins_centered, normalization, tail_percent=1, constant_percent=sys_width) 
    #make dict
    hist_dict[(reg, mass, dbs)] = {} 
    hist_dict[(reg, mass, dbs)]['mc'] =  hist
    hist_dict[(reg, mass, dbs)]['fit'] = fit_hist

In [ ]:
def draw_mass_ratio(top, bottom, tmass, treg):
    for (reg, mass, dbs), plot_dict in hist_dict.items():
        if mass not in [tmass]: continue
        color = color_fader('red', 'blue', float(1-dbs)/(1-0.04))
        #if dbs!=0.04: continue
        linestyle = {0.04:"-",0.5:"--",1:":"}[dbs]
        if reg!=treg: continue
        hist = plot_dict['mc']
        fit_hist = plot_dict['fit']
        sm = bff_signal_model(obs, mu=mass)
        width = sm.sigma().numpy()
        #remove sys for clarity 
        hist_norm = hist.calc_ratio(np.sum(hist.nominal))
        hist_norm.up = hist_norm.up*0
        hist_norm.down = hist_norm.down*0
        hist_norm.bin_edges = np.linspace(-7, 7,50)
        
        hist_norm.make_density_hist().draw(top, color=color, error_scale=1,linestyle=linestyle, label='{} {}'.format(mass, dbs))
        #calculate ratio compared to 0.04 point
        ratio_hist = hist_dict[(reg,tmass,0.04)]['mc'].nominal
        ratio_hist = ratio_hist/np.sum(ratio_hist)
        ratio = hist_norm.calc_ratio(ratio_hist)
        
        ratio.draw(bottom, color=color, error_scale=1,linestyle=linestyle)
        #fit_hist.make_density_hist().draw(ax, color='blue', label='fits')
    top.set_yscale('log')
    #bottom.set_yscale('log')
    top.legend(title=treg)
    top.set_ylabel('Events per GeV')
    bottom.set_xlabel('DiLepMass [GeV]')
    bottom.set_ylim(bottom=-0, top=2)
    top.set_ylim(bottom=1e-4)
    cms_format_fig(era, top)

In [ ]:
#non-density
fig, axs = plt.subplots(2,3,figsize=(30,10))
draw_mass_ratio(axs[0][0], axs[1][0], 125,"SR1")
draw_mass_ratio(axs[0][1], axs[1][1], 350,"SR1")
draw_mass_ratio(axs[0][2], axs[1][2], 500,"SR1")
fig.savefig('fits/bff/center_binning_dbs_ratio_{}_{}_{}.png'.format(binning_type, era, "SR1", mass))

In [ ]:
#non-density
fig, axs = plt.subplots(2,3,figsize=(30,10))
    
draw_mass_ratio(axs[0][0], axs[1][0], 125,"SR2")
draw_mass_ratio(axs[0][1], axs[1][1], 350,"SR2")
draw_mass_ratio(axs[0][2], axs[1][2], 500,"SR2")
fig.savefig('fits/bff/center_binning_dbs_ratio_{}_{}_{}.png'.format(binning_type, era, "SR2", mass))